In [ ]:

import sys
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, when, isnull, trim, avg, count, sum
from pyspark.sql.types import StringType, IntegerType
bucketNameTrusted = "atividade5-trusted/"
bucketNameDelivery = "atividade5-delivery/"

pastaBancos = "bancos/"
pastaEmpregados = "empregados/"
pastaReclamacoes = "reclamacoes/"
df_bancos = spark.read.parquet('s3://'+bucketNameTrusted+pastaBancos)
df_empregados = spark.read.parquet('s3://'+bucketNameTrusted+pastaEmpregados)
df_reclamacoes = spark.read.parquet('s3://'+bucketNameTrusted+pastaReclamacoes)
columns = ['segmento','cnpj','nome',col("geral").cast('float').alias("geral"),col("remuneracao_e_beneficios").cast('float').alias("remuneracao_e_beneficios")]

df_bancos_empregados = df_bancos.join(df_empregados,df_bancos.nome ==  df_empregados.nome_emp,"inner").select(columns).distinct()
columns = ['segmento','cnpj','nome_rec','ano','trimestre',col("indice").cast('float').alias("indice"),\
           col("qtd_reclamacoes").cast('int').alias("qtd_reclamacoes"),col("qtd_clientes").cast('int').alias("qtd_clientes")]

df_bancos_reclamacoes = df_bancos.join(df_reclamacoes,df_bancos.nome ==  df_reclamacoes.nome_rec,"inner").select(columns)
df_bancos_reclamacoes = df_bancos_reclamacoes.groupBy("nome_rec").agg(avg("indice").alias("indice"), sum("qtd_reclamacoes").alias("qtd_reclamacoes") \
                      , avg("qtd_clientes").alias("qtd_clientes"))
columns = [col("nome").alias("nome_banco"),"cnpj",col("segmento").alias("classificacao_banco"),\
           col("qtd_clientes"),col("indice").alias("indice_de_reclamacoes"),\
           col("qtd_reclamacoes"),col("geral").alias("indice_satisfacao_geral"),\
           col("remuneracao_e_beneficios").alias("indice_satisfacao_salarios")]

df_fato = df_bancos_empregados.join(df_bancos_reclamacoes,df_bancos_empregados.nome ==  df_bancos_reclamacoes.nome_rec,"inner").select(columns).distinct()
df_fato.write.format("hive").mode("append").saveAsTable("bancos_classif.fato")
df_fato.write.mode("overwrite").parquet('s3://'+bucketNameDelivery+"fato/")
job.commit()